### traindata 1000개씩(총9000개) - 모두 30초 미만임을 확인✔️
- 4세: 1109*
- 5세: 1068*
- 6세: 2080*
- 7세: 2196*
- 8세: 1088*
- 9세: 1025 *
- 10세: 2336*
- 11세: 2370*
- 12세: 1058*
### validationdata 125개씩(총1125개) - 모두 30초 미만임을 확인✔️
- 4세: 0055*
- 5세: 0002*
- 6세: 0096*
- 7세: 0901*
- 8세: 0219*
- 9세: 0018*
- 10세: 0009*
- 11세: 0422*
- 12세: 0034*
### testdata 125개씩(총1125개) - 모두 30초 미만임을 확인✔️
- validation데이터에 4세 데이터 없어서 train 데이터에서 사용할
- 4세: 1079(train)
- validation220106 폴더에서
- 5세: 4858*
- 6세: 5206*
- 7세: 6124*
- 8세: 7933*
- 9세: 7894*
- 10세: 6152*
- 11세: 8573*
- 12세: 8838*
###8:1:1 (총11250개)

In [ ]:
import os
import json
import pandas as pd

# 1️⃣ 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2️⃣ 라벨링 데이터 기본 경로
BASE_DIR = "/content/drive/MyDrive/childrenvoice/trainingdata/labellingdata"

# 3️⃣ 확인할 화자 ID 목록 (나이랑 매핑했던 애들)
speaker_ids = [
    "1109",  # 4세
    "1068",  # 5세
    "2080",  # 6세
    "2196",  # 7세
    "1088",  # 8세
    "1025",  # 9세
    "2336",  # 10세
    "2370",  # 11세
    "1058",  # 12세
]

violations = []      # 30초 이상 파일 저장
summary_rows = []    # 화자별 최대 길이 요약

for speaker_id in speaker_ids:
    speaker_dir = os.path.join(BASE_DIR, speaker_id)
    if not os.path.isdir(speaker_dir):
        print(f"⚠️ 폴더 없음: {speaker_dir}")
        continue

    max_len = 0.0
    min_len = float("inf")  # 최소값 초기화
    file_count = 0

    for fname in os.listdir(speaker_dir):
        if not fname.endswith(".json"):
            continue

        json_path = os.path.join(speaker_dir, fname)
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            file_length = float(data["File"]["FileLength"])  # 전체 wav 길이(초)
            file_count += 1

            # 최대 길이 갱신
            if file_length > max_len:
                max_len = file_length

            # 최소 길이 갱신
            if file_length < min_len:
                min_len = file_length

            # 30초 이상 파일 기록
            if file_length >= 30.0:
                violations.append({
                    "speaker_id": speaker_id,
                    "file_name": data["File"]["FileName"],
                    "file_length_sec": file_length,
                })

        except Exception as e:
            print(f"❌ 오류: {json_path} / {e}")

    summary_rows.append({
        "speaker_id": speaker_id,
        "file_count": file_count,
        "min_file_length_sec": round(min_len, 3),
        "max_file_length_sec": round(max_len, 3),
    })

# 4️⃣ 화자별 요약 출력
summary_df = pd.DataFrame(summary_rows)
print("🔹 화자별 파일 개수 / 최소 길이 / 최대 길이(초):")
print(summary_df)

# 5️⃣ 30초 이상 파일 체크
if len(violations) == 0:
    print("\n✅ 모든 화자의 모든 음성이 30초 미만입니다.")
else:
    print("\n⚠️ 30초 이상인 파일들이 있습니다:")
    print(pd.DataFrame(violations))

🔹 화자별 파일 개수 / 최소 길이 / 최대 길이(초):
  speaker_id  file_count  min_file_length_sec  max_file_length_sec
0       1109        1000                2.048                9.216
1       1068        1000                3.712                7.616
2       2080        1000                2.368                9.792
3       2196        1000                3.072               13.312
4       1088        1000                2.560               14.848
5       1025        1000                2.816                9.856
6       2336        1000                4.096               11.264
7       2370        1000                3.584               11.648
8       1058        1000                3.072               11.264

✅ 모든 화자의 모든 음성이 30초 미만입니다.


###validation data 확인

In [ ]:
# 2️⃣ 라벨링 데이터 경로 (training이랑 동일)
BASE_DIR = "/content/drive/MyDrive/childrenvoice/trainingdata/labellingdata"

# 3️⃣ 이번에 확인할 화자들 (validation 화자들)
age_speaker_val = {
    4:  "0055",
    5:  "0209",
    6:  "0153",
    7:  "0901",
    8:  "0219",
    9:  "0018",
    10: "0009",
    11: "0422",
    12: "0034",
}

summary_rows = []   # 화자별 통계
violations = []     # 30초 이상 파일

for age, speaker_id in age_speaker_val.items():
    speaker_dir = os.path.join(BASE_DIR, speaker_id)

    if not os.path.isdir(speaker_dir):
        print(f"⚠️ 폴더 없음: {speaker_dir}")
        continue

    max_len = 0.0
    min_len = float("inf")
    file_count = 0

    for fname in os.listdir(speaker_dir):
        if not fname.endswith(".json"):
            continue

        json_path = os.path.join(speaker_dir, fname)

        try:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # 전체 wav 길이 (초)
            length = float(data["File"]["FileLength"])
            file_count += 1

            # 최소 / 최대 갱신
            if length > max_len:
                max_len = length
            if length < min_len:
                min_len = length

            # 30초 이상이면 기록
            if length >= 30.0:
                violations.append({
                    "age": age,
                    "speaker_id": speaker_id,
                    "file_name": data["File"]["FileName"],
                    "file_length_sec": length,
                })

        except Exception as e:
            print(f"❌ 오류: {json_path} / {e}")

    summary_rows.append({
        "age": age,
        "speaker_id": speaker_id,
        "file_count": file_count,
        "min_file_length_sec": round(min_len, 3),
        "max_file_length_sec": round(max_len, 3),
    })

# 4️⃣ 화자별 요약 테이블
summary_df = pd.DataFrame(summary_rows).sort_values("age")
print("🔹 화자별 파일 개수 / 최소 길이 / 최대 길이(초):")
print(summary_df)

# 5️⃣ 30초 이상 파일 존재 여부
if len(violations) == 0:
    print("\n✅ 이 9명 화자의 모든 음성 파일이 30초 미만입니다.")
else:
    print("\n⚠️ 30초 이상인 파일이 있습니다:")
    viol_df = pd.DataFrame(violations).sort_values(["age", "speaker_id"])
    print(viol_df)

⚠️ 폴더 없음: /content/drive/MyDrive/childrenvoice/trainingdata/labellingdata/0209
🔹 화자별 파일 개수 / 최소 길이 / 최대 길이(초):
   age speaker_id  file_count  min_file_length_sec  max_file_length_sec
0    4       0055         684                2.304               13.312
1    6       0153         972                2.048               15.360
2    7       0901         433                4.352               12.800
3    8       0219         401                2.816               12.416
4    9       0018         154                4.096               15.360
5   10       0009         178                5.120               15.360
6   11       0422         154                3.072               18.432
7   12       0034         255                6.016               13.632

✅ 이 9명 화자의 모든 음성 파일이 30초 미만입니다.


###testdata 고르기
- 나이별 125개씩
- ai-hub가 선정한 validation데이터셋을 test데이터로 사용할 예정
- 2022-01-06 폴더에서 선택 (가장 최신 폴더)

In [ ]:
# ============================================================
# 1) Google Drive 마운트
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# ============================================================
# 2) 경로/설정
# ============================================================
import os, re, json, csv
from glob import glob

# 🔹 valdata 라벨링 경로
BASE_DIR = '/content/drive/MyDrive/childrenvoice/valdata/labellingdata'

# ============================================================
# 3) Age 추출 유틸 (JSON 어디에 있어도 "Age" 먼저 찾기)
# ============================================================
AGE_KEYS = ['Age', 'age', 'AGE']

def normalize_age(v):
    """문자열/숫자형 Age를 정수로 정규화 (0~99 사이만 유효)"""
    if v is None:
        return None
    if isinstance(v, (int, float)):
        a = int(v)
        return a if 0 <= a <= 99 else None
    s = str(v).strip()
    m = re.search(r'(\d{1,2})', s)  # "6", "06", "만 6세" → 6
    if not m:
        return None
    a = int(m.group(1))
    return a if 0 <= a <= 99 else None

def extract_age_from_json_obj(d):
    """중첩 dict/list 구조에서 Age 계열 키를 깊이우선 탐색"""
    stack = [d]
    while stack:
        cur = stack.pop()
        if isinstance(cur, dict):
            # 1) 현재 dict에서 Age 키 찾기
            for k in AGE_KEYS:
                if k in cur:
                    a = normalize_age(cur[k])
                    if a is not None:
                        return a
            # 2) 값들 안으로 계속 내려가기
            for v in cur.values():
                if isinstance(v, (dict, list)):
                    stack.append(v)
        elif isinstance(cur, list):
            for it in cur:
                if isinstance(it, (dict, list)):
                    stack.append(it)
    return None

def extract_age_from_speaker_dir(sp_dir):
    """
    화자 폴더 내 JSON들 중 최대 5개까지 확인해서
    Age 하나라도 발견되면 그 값을 반환
    """
    json_files = sorted(glob(os.path.join(sp_dir, '**', '*.json'), recursive=True))
    for jf in json_files[:5]:  # 과한 I/O 방지용
        try:
            with open(jf, 'r', encoding='utf-8') as fh:
                data = json.load(fh)
            age = extract_age_from_json_obj(data)
            if age is not None:
                return age
        except Exception:
            pass
    return None

# ============================================================
# 4) 화자 폴더 순회 & 라벨링파일 개수/나이 수집
# ============================================================
speaker_dirs = sorted([
    p for p in glob(os.path.join(BASE_DIR, '*'))
    if os.path.isdir(p)
])

rows = []  # (speaker_id, age, label_file_count)

for sp_dir in speaker_dirs:
    sp_id = os.path.basename(sp_dir)

    # 🔹 해당 화자 폴더(하위 포함)의 라벨링파일 개수 (여기서는 .json 기준)
    label_files = glob(os.path.join(sp_dir, '**', '*.json'), recursive=True)
    label_count = len(label_files)

    # 🔹 Age 추출 (해당 화자 폴더 안 JSON들에서)
    age = extract_age_from_speaker_dir(sp_dir)

    rows.append((sp_id, age, label_count))

# ============================================================
# 5) 정렬: age 오름차순, 같은 나이 안에서는 label_file_count 내림차순
#     (age 없는 화자는 맨 뒤)
# ============================================================
def sort_key(t):
    sp_id, age, cnt = t
    return (age is None, age if age is not None else 999, -cnt, sp_id)

rows_sorted = sorted(rows, key=sort_key)

# ============================================================
# 6) CSV 저장 + 요약 출력
# ============================================================
out_csv = os.path.join(BASE_DIR, 'val_speaker_age_labelcount.csv')
with open(out_csv, 'w', newline='', encoding='utf-8') as f:
    w = csv.writer(f)
    w.writerow(['speaker_id', 'age', 'label_file_count'])  # = 발화(음성) 개수로 보면 됨
    for r in rows_sorted:
        w.writerow(r)

print("✅ 저장 완료 →", out_csv)
print("총 화자 수:", len(rows_sorted))

print("\n[미리보기 상위 15]")
for r in rows_sorted[:15]:
    print(r)

missing_age = sum(1 for _, age, _ in rows_sorted if age is None)
print("\n나이 누락 화자 수:", missing_age)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 저장 완료 → /content/drive/MyDrive/childrenvoice/valdata/labellingdata/val_speaker_age_labelcount.csv
총 화자 수: 410

[미리보기 상위 15]
('4411', 5, 1000)
('4677', 5, 1000)
('4462', 5, 998)
('4683', 5, 995)
('7951', 5, 991)
('8018', 5, 827)
('8204', 5, 421)
('6084', 5, 250)
('8743', 5, 208)
('5098', 5, 206)
('8686', 5, 187)
('4858', 5, 132)
('4249', 5, 110)
('4659', 5, 101)
('4374', 5, 86)

나이 누락 화자 수: 0


In [ ]:
# ============================================================
# 1) Google Drive 마운트
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# ============================================================
# 2) 경로/설정
# ============================================================
import os, re, json, csv
from glob import glob

# 🔹 valdata 라벨링 경로
BASE_DIR = '/content/drive/MyDrive/childrenvoice/valdata/labellingdata2'

# ============================================================
# 3) Age 추출 유틸 (JSON 어디에 있어도 "Age" 먼저 찾기)
# ============================================================
AGE_KEYS = ['Age', 'age', 'AGE']

def normalize_age(v):
    """문자열/숫자형 Age를 정수로 정규화 (0~99 사이만 유효)"""
    if v is None:
        return None
    if isinstance(v, (int, float)):
        a = int(v)
        return a if 0 <= a <= 99 else None
    s = str(v).strip()
    m = re.search(r'(\d{1,2})', s)  # "6", "06", "만 6세" → 6
    if not m:
        return None
    a = int(m.group(1))
    return a if 0 <= a <= 99 else None

def extract_age_from_json_obj(d):
    """중첩 dict/list 구조에서 Age 계열 키를 깊이우선 탐색"""
    stack = [d]
    while stack:
        cur = stack.pop()
        if isinstance(cur, dict):
            # 1) 현재 dict에서 Age 키 찾기
            for k in AGE_KEYS:
                if k in cur:
                    a = normalize_age(cur[k])
                    if a is not None:
                        return a
            # 2) 값들 안으로 계속 내려가기
            for v in cur.values():
                if isinstance(v, (dict, list)):
                    stack.append(v)
        elif isinstance(cur, list):
            for it in cur:
                if isinstance(it, (dict, list)):
                    stack.append(it)
    return None

def extract_age_from_speaker_dir(sp_dir):
    """
    화자 폴더 내 JSON들 중 최대 5개까지 확인해서
    Age 하나라도 발견되면 그 값을 반환
    """
    json_files = sorted(glob(os.path.join(sp_dir, '**', '*.json'), recursive=True))
    for jf in json_files[:5]:  # 과한 I/O 방지용
        try:
            with open(jf, 'r', encoding='utf-8') as fh:
                data = json.load(fh)
            age = extract_age_from_json_obj(data)
            if age is not None:
                return age
        except Exception:
            pass
    return None

# ============================================================
# 4) 화자 폴더 순회 & 라벨링파일 개수/나이 수집
# ============================================================
speaker_dirs = sorted([
    p for p in glob(os.path.join(BASE_DIR, '*'))
    if os.path.isdir(p)
])

rows = []  # (speaker_id, age, label_file_count)

for sp_dir in speaker_dirs:
    sp_id = os.path.basename(sp_dir)

    # 🔹 해당 화자 폴더(하위 포함)의 라벨링파일 개수 (여기서는 .json 기준)
    label_files = glob(os.path.join(sp_dir, '**', '*.json'), recursive=True)
    label_count = len(label_files)

    # 🔹 Age 추출 (해당 화자 폴더 안 JSON들에서)
    age = extract_age_from_speaker_dir(sp_dir)

    rows.append((sp_id, age, label_count))

# ============================================================
# 5) 정렬: age 오름차순, 같은 나이 안에서는 label_file_count 내림차순
#     (age 없는 화자는 맨 뒤)
# ============================================================
def sort_key(t):
    sp_id, age, cnt = t
    return (age is None, age if age is not None else 999, -cnt, sp_id)

rows_sorted = sorted(rows, key=sort_key)

# ============================================================
# 6) CSV 저장 + 요약 출력
# ============================================================
out_csv = os.path.join(BASE_DIR, 'val_speaker_age_labelcount2.csv')
with open(out_csv, 'w', newline='', encoding='utf-8') as f:
    w = csv.writer(f)
    w.writerow(['speaker_id', 'age', 'label_file_count'])  # = 발화(음성) 개수로 보면 됨
    for r in rows_sorted:
        w.writerow(r)

print("✅ 저장 완료 →", out_csv)
print("총 화자 수:", len(rows_sorted))

print("\n[미리보기 상위 15]")
for r in rows_sorted[:15]:
    print(r)

missing_age = sum(1 for _, age, _ in rows_sorted if age is None)
print("\n나이 누락 화자 수:", missing_age)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 저장 완료 → /content/drive/MyDrive/childrenvoice/valdata/labellingdata2/val_speaker_age_labelcount2.csv
총 화자 수: 68

[미리보기 상위 15]
('7951', 5, 8)
('8018', 5, 1)
('7960', 6, 56)
('7995', 6, 30)
('7985', 7, 49)
('7947', 7, 12)
('7933', 8, 63)
('8027', 8, 23)
('7986', 8, 10)
('7942', 9, 239)
('7964', 9, 107)
('7495', 9, 74)
('7905', 9, 27)
('8068', 9, 5)
('6679', 10, 828)

나이 누락 화자 수: 0


In [ ]:
# ============================================================
# 1) Google Drive 마운트
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# ============================================================
# 2) 경로/설정
# ============================================================
import os, re, json, csv
from glob import glob

# 🔹 valdata 라벨링 경로
BASE_DIR = '/content/drive/MyDrive/childrenvoice/valdata/labellingdata3'

# ============================================================
# 3) Age 추출 유틸 (JSON 어디에 있어도 "Age" 먼저 찾기)
# ============================================================
AGE_KEYS = ['Age', 'age', 'AGE']

def normalize_age(v):
    """문자열/숫자형 Age를 정수로 정규화 (0~99 사이만 유효)"""
    if v is None:
        return None
    if isinstance(v, (int, float)):
        a = int(v)
        return a if 0 <= a <= 99 else None
    s = str(v).strip()
    m = re.search(r'(\d{1,2})', s)  # "6", "06", "만 6세" → 6
    if not m:
        return None
    a = int(m.group(1))
    return a if 0 <= a <= 99 else None

def extract_age_from_json_obj(d):
    """중첩 dict/list 구조에서 Age 계열 키를 깊이우선 탐색"""
    stack = [d]
    while stack:
        cur = stack.pop()
        if isinstance(cur, dict):
            # 1) 현재 dict에서 Age 키 찾기
            for k in AGE_KEYS:
                if k in cur:
                    a = normalize_age(cur[k])
                    if a is not None:
                        return a
            # 2) 값들 안으로 계속 내려가기
            for v in cur.values():
                if isinstance(v, (dict, list)):
                    stack.append(v)
        elif isinstance(cur, list):
            for it in cur:
                if isinstance(it, (dict, list)):
                    stack.append(it)
    return None

def extract_age_from_speaker_dir(sp_dir):
    """
    화자 폴더 내 JSON들 중 최대 5개까지 확인해서
    Age 하나라도 발견되면 그 값을 반환
    """
    json_files = sorted(glob(os.path.join(sp_dir, '**', '*.json'), recursive=True))
    for jf in json_files[:5]:  # 과한 I/O 방지용
        try:
            with open(jf, 'r', encoding='utf-8') as fh:
                data = json.load(fh)
            age = extract_age_from_json_obj(data)
            if age is not None:
                return age
        except Exception:
            pass
    return None

# ============================================================
# 4) 화자 폴더 순회 & 라벨링파일 개수/나이 수집
# ============================================================
speaker_dirs = sorted([
    p for p in glob(os.path.join(BASE_DIR, '*'))
    if os.path.isdir(p)
])

rows = []  # (speaker_id, age, label_file_count)

for sp_dir in speaker_dirs:
    sp_id = os.path.basename(sp_dir)

    # 🔹 해당 화자 폴더(하위 포함)의 라벨링파일 개수 (여기서는 .json 기준)
    label_files = glob(os.path.join(sp_dir, '**', '*.json'), recursive=True)
    label_count = len(label_files)

    # 🔹 Age 추출 (해당 화자 폴더 안 JSON들에서)
    age = extract_age_from_speaker_dir(sp_dir)

    rows.append((sp_id, age, label_count))

# ============================================================
# 5) 정렬: age 오름차순, 같은 나이 안에서는 label_file_count 내림차순
#     (age 없는 화자는 맨 뒤)
# ============================================================
def sort_key(t):
    sp_id, age, cnt = t
    return (age is None, age if age is not None else 999, -cnt, sp_id)

rows_sorted = sorted(rows, key=sort_key)

# ============================================================
# 6) CSV 저장 + 요약 출력
# ============================================================
out_csv = os.path.join(BASE_DIR, 'val_speaker_age_labelcount3.csv')
with open(out_csv, 'w', newline='', encoding='utf-8') as f:
    w = csv.writer(f)
    w.writerow(['speaker_id', 'age', 'label_file_count'])  # = 발화(음성) 개수로 보면 됨
    for r in rows_sorted:
        w.writerow(r)

print("✅ 저장 완료 →", out_csv)
print("총 화자 수:", len(rows_sorted))

print("\n[미리보기 상위 15]")
for r in rows_sorted[:15]:
    print(r)

missing_age = sum(1 for _, age, _ in rows_sorted if age is None)
print("\n나이 누락 화자 수:", missing_age)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 저장 완료 → /content/drive/MyDrive/childrenvoice/valdata/labellingdata3/val_speaker_age_labelcount3.csv
총 화자 수: 226

[미리보기 상위 15]
('2854', 5, 22)
('4430', 5, 9)
('1981', 5, 5)
('2644', 5, 5)
('7511', 6, 201)
('3775', 6, 48)
('3679', 6, 40)
('2846', 6, 33)
('3571', 6, 28)
('4177', 6, 17)
('3937', 6, 14)
('3625', 6, 4)
('4975', 7, 550)
('5050', 7, 287)
('4375', 7, 192)

나이 누락 화자 수: 0


In [ ]:
# ============================================================
# 1) Google Drive 마운트
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# ============================================================
# 2) 경로/설정
# ============================================================
import os, re, json, csv
from glob import glob

# 🔹 valdata 라벨링 경로
BASE_DIR = '/content/drive/MyDrive/childrenvoice/valdata/labellingdata4'

# ============================================================
# 3) Age 추출 유틸 (JSON 어디에 있어도 "Age" 먼저 찾기)
# ============================================================
AGE_KEYS = ['Age', 'age', 'AGE']

def normalize_age(v):
    """문자열/숫자형 Age를 정수로 정규화 (0~99 사이만 유효)"""
    if v is None:
        return None
    if isinstance(v, (int, float)):
        a = int(v)
        return a if 0 <= a <= 99 else None
    s = str(v).strip()
    m = re.search(r'(\d{1,2})', s)  # "6", "06", "만 6세" → 6
    if not m:
        return None
    a = int(m.group(1))
    return a if 0 <= a <= 99 else None

def extract_age_from_json_obj(d):
    """중첩 dict/list 구조에서 Age 계열 키를 깊이우선 탐색"""
    stack = [d]
    while stack:
        cur = stack.pop()
        if isinstance(cur, dict):
            # 1) 현재 dict에서 Age 키 찾기
            for k in AGE_KEYS:
                if k in cur:
                    a = normalize_age(cur[k])
                    if a is not None:
                        return a
            # 2) 값들 안으로 계속 내려가기
            for v in cur.values():
                if isinstance(v, (dict, list)):
                    stack.append(v)
        elif isinstance(cur, list):
            for it in cur:
                if isinstance(it, (dict, list)):
                    stack.append(it)
    return None

def extract_age_from_speaker_dir(sp_dir):
    """
    화자 폴더 내 JSON들 중 최대 5개까지 확인해서
    Age 하나라도 발견되면 그 값을 반환
    """
    json_files = sorted(glob(os.path.join(sp_dir, '**', '*.json'), recursive=True))
    for jf in json_files[:5]:  # 과한 I/O 방지용
        try:
            with open(jf, 'r', encoding='utf-8') as fh:
                data = json.load(fh)
            age = extract_age_from_json_obj(data)
            if age is not None:
                return age
        except Exception:
            pass
    return None

# ============================================================
# 4) 화자 폴더 순회 & 라벨링파일 개수/나이 수집
# ============================================================
speaker_dirs = sorted([
    p for p in glob(os.path.join(BASE_DIR, '*'))
    if os.path.isdir(p)
])

rows = []  # (speaker_id, age, label_file_count)

for sp_dir in speaker_dirs:
    sp_id = os.path.basename(sp_dir)

    # 🔹 해당 화자 폴더(하위 포함)의 라벨링파일 개수 (여기서는 .json 기준)
    label_files = glob(os.path.join(sp_dir, '**', '*.json'), recursive=True)
    label_count = len(label_files)

    # 🔹 Age 추출 (해당 화자 폴더 안 JSON들에서)
    age = extract_age_from_speaker_dir(sp_dir)

    rows.append((sp_id, age, label_count))

# ============================================================
# 5) 정렬: age 오름차순, 같은 나이 안에서는 label_file_count 내림차순
#     (age 없는 화자는 맨 뒤)
# ============================================================
def sort_key(t):
    sp_id, age, cnt = t
    return (age is None, age if age is not None else 999, -cnt, sp_id)

rows_sorted = sorted(rows, key=sort_key)

# ============================================================
# 6) CSV 저장 + 요약 출력
# ============================================================
out_csv = os.path.join(BASE_DIR, 'val_speaker_age_labelcount4.csv')
with open(out_csv, 'w', newline='', encoding='utf-8') as f:
    w = csv.writer(f)
    w.writerow(['speaker_id', 'age', 'label_file_count'])  # = 발화(음성) 개수로 보면 됨
    for r in rows_sorted:
        w.writerow(r)

print("✅ 저장 완료 →", out_csv)
print("총 화자 수:", len(rows_sorted))

print("\n[미리보기 상위 15]")
for r in rows_sorted[:15]:
    print(r)

missing_age = sum(1 for _, age, _ in rows_sorted if age is None)
print("\n나이 누락 화자 수:", missing_age)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 저장 완료 → /content/drive/MyDrive/childrenvoice/valdata/labellingdata4/val_speaker_age_labelcount4.csv
총 화자 수: 36

[미리보기 상위 15]
('8062', 6, 30)
('8100', 7, 5)
('8015', 8, 36)
('8017', 9, 35)
('8038', 9, 14)
('7888', 10, 127)
('7728', 10, 102)
('7969', 10, 101)
('7532', 10, 69)
('7547', 10, 54)
('7574', 10, 54)
('7856', 10, 51)
('7810', 10, 39)
('7973', 10, 37)
('7831', 10, 26)

나이 누락 화자 수: 0


###선택한 testdata 길이 확인

In [6]:
import os
import json
import pandas as pd

# 1️⃣ 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2️⃣ test 데이터 최상위 경로
BASE_DIR = "/content/drive/MyDrive/childrenvoice/testdata"

# 3️⃣ 이번에 확인할 test 화자들 (나이 → 화자 ID)
age_speaker_test = {
    4:  "1079",
    5:  "4858",
    6:  "5206",
    7:  "6124",
    8:  "7933",
    9:  "7894",
    10: "6152",
    11: "8573",
    12: "8838",
}

summary_rows = []   # 화자별 통계
violations = []     # 30초 이상 파일

for age, speaker_id in age_speaker_test.items():

    # 🔹 실제 경로: testdata/age{age}/labelling/{speaker_id}
    speaker_dir = os.path.join(BASE_DIR, f"age{age}", "labelling", speaker_id)

    if not os.path.isdir(speaker_dir):
        print(f"⚠️ 폴더 없음: {speaker_dir}")
        continue

    max_len = 0.0
    min_len = float("inf")
    file_count = 0

    for fname in os.listdir(speaker_dir):
        if not fname.endswith(".json"):
            continue

        json_path = os.path.join(speaker_dir, fname)

        try:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # 전체 wav 길이 (초)
            length = float(data["File"]["FileLength"])
            file_count += 1

            # 최소 / 최대 갱신
            if length > max_len:
                max_len = length
            if length < min_len:
                min_len = length

            # 30초 이상이면 기록
            if length >= 30.0:
                violations.append({
                    "age": age,
                    "speaker_id": speaker_id,
                    "file_name": data["File"]["FileName"],
                    "file_length_sec": length,
                })

        except Exception as e:
            print(f"❌ 오류: {json_path} / {e}")

    summary_rows.append({
        "age": age,
        "speaker_id": speaker_id,
        "file_count": file_count,
        "min_file_length_sec": round(min_len, 3),
        "max_file_length_sec": round(max_len, 3),
    })

# 4️⃣ 화자별 요약 테이블
summary_df = pd.DataFrame(summary_rows).sort_values("age")
print("🔹 (test) 화자별 파일 개수 / 최소 길이 / 최대 길이(초):")
print(summary_df)

# 5️⃣ 30초 이상 파일 존재 여부
if len(violations) == 0:
    print("\n✅ 이 9명(test) 화자의 모든 음성 파일이 30초 미만입니다.")
else:
    print("\n⚠️ 30초 이상인 파일이 있습니다:")
    viol_df = pd.DataFrame(violations).sort_values(["age", "speaker_id"])
    print(viol_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔹 (test) 화자별 파일 개수 / 최소 길이 / 최대 길이(초):
   age speaker_id  file_count  min_file_length_sec  max_file_length_sec
0    4       1079         991                2.112                7.936
1    5       4858         132                2.816                6.912
2    6       5206         133                2.304                8.064
3    7       6124         130                5.120               20.480
4    8       7933         136                3.072               10.240
5    9       7894         129                2.048                7.680
6   10       6152         137                5.056               15.808
7   11       8573         128                5.632               11.904
8   12       8838         199                4.096               12.288

✅ 이 9명(test) 화자의 모든 음성 파일이 30초 미만입니다.
